<a href="https://colab.research.google.com/github/brianzkee/gsis_arrears_auto/blob/main/gsis_arrears_auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# List of tuples with the Lastname and Firstname pairs to filter, add the names in this pattern
filter_list = [
    ('Lastname1', 'Firstname1'),
    ('Lastname2', 'Firstname2')
]

# Load the CSV file to examine its contents, change the path to your files
file_path = '/content/drive/MyDrive/Colab Notebooks/GSIS Arrears Automation/upr_202408_DEPED  ELEM CATB 1706 M-Z.csv'
template_path = '/content/drive/MyDrive/Colab Notebooks/GSIS Arrears Automation/template.xlsx'
output_base_path = '/content/drive/MyDrive/Colab Notebooks/GSIS Arrears Automation/computed_output_sheets_batch'  # Base path for output files

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Define the columns we are interested in
columns_to_select = [
    'BPNO', 'Lastname', 'Firstname', 'MI', 'Prefix', 'Appellation', 'Birth Date',
    'CRN', 'Period Start', 'Monthly Salary', 'Unpaid PS', 'Unpaid GS', 'Unpaid EC'
]

# Skip initial rows and rename the columns to match desired headers
df_clean = pd.read_csv(file_path, usecols=[1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 19, 20, 21])

# Rename the columns to desired names
df_clean.columns = columns_to_select

# Reorder the columns manually based on your desired order
desired_order = [
    'BPNO', 'Lastname', 'Firstname', 'MI', 'Prefix', 'Appellation', 'Birth Date',
    'CRN', 'Monthly Salary', 'Period Start','Unpaid PS', 'Unpaid GS', 'Unpaid EC'
]
df_clean = df_clean[desired_order]

# Convert the list into a DataFrame for easier filtering
filter_df = pd.DataFrame(filter_list, columns=['Lastname', 'Firstname'])

# Filter the dataframe based on the 'Lastname' and 'Firstname' columns
df_filtered = pd.merge(df_clean, filter_df, on=['Lastname', 'Firstname'])

# Define the columns to check for discrepancies
columns_to_check = ['Unpaid PS', 'Unpaid GS', 'Unpaid EC']

# List to store indices of rows to be deleted
rows_to_drop = []

# Iterate through the dataframe, starting from the second row
for i in range(1, len(df_filtered)):
    if (df_filtered.loc[i, 'Unpaid PS'] < 0) or (df_filtered.loc[i, 'Unpaid GS'] < 0) or (df_filtered.loc[i, 'Unpaid EC'] < 0):
        # Add the values from the previous row to the current row
        for col in columns_to_check:
            df_filtered.loc[i, col] += df_filtered.loc[i - 1, col]
        # Mark the previous row for deletion
        rows_to_drop.append(i - 1)

# Drop the marked rows
df_filtered.drop(rows_to_drop, inplace=True)
df_filtered.reset_index(drop=True, inplace=True)

# Rename df_filtered to df_computed for readability
df_computed = df_filtered.copy()

# Create a separate column for grouping purposes (Period Start in datetime format) without modifying the original
df_computed['Period Start Group'] = pd.to_datetime(df_computed['Period Start'], errors='coerce')

# Load the template workbook
wb = None  # We will dynamically create workbooks
worksheet_count = 0
batch_number = 1

# Group by the new 'Period Start Group' column (using Year and Month) without modifying the original 'Period Start'
for (year, month), group in df_computed.groupby([df_computed['Period Start Group'].dt.year, df_computed['Period Start Group'].dt.month]):
    # Initialize a new workbook if none exists or if we've reached the 36 worksheet limit
    if wb is None or worksheet_count >= 36:
        if wb is not None:
            # Save the previous workbook before creating a new one
            output_path = f"{output_base_path}{batch_number}.xlsx"
            # Remove the sample sheet (optional, if you want to keep it, skip this step)
            if 'year-month' in wb.sheetnames:
              wb.remove(wb['year-month'])

            wb.save(output_path)
            print(f"Batch {batch_number} saved to {output_path}")
            batch_number += 1
        # Start a new workbook
        wb = load_workbook(template_path)
        worksheet_count = 0

    df_name = f"{year}{month:02d}"  # Format the dataframe name as 'yyyy-mm'

    # Duplicate the template worksheet
    ws_template = wb['year-month']  # Assuming 'year-month' is the name of the template worksheet
    new_sheet = wb.copy_worksheet(ws_template)
    new_sheet.title = df_name

    # Set Due Month value in cell B3
    new_sheet['B3'] = f"{month:02d}/{year}"

    # Drop the 'Period Start Group' column from the group before writing to Excel
    group = group.drop(columns=['Period Start Group'])

    # Remove column headers from the dataframe and start inserting from row 6
    for r_idx, row in enumerate(dataframe_to_rows(group, index=False, header=False), start=6):
        for c_idx, value in enumerate(row, start=1):
            new_sheet.cell(row=r_idx, column=c_idx, value=value)

    worksheet_count += 1

# Save the last workbook (if it has any worksheets)
if wb is not None:
    output_path = f"{output_base_path}{batch_number}.xlsx"

    # Remove the sample sheet (optional, if you want to keep it, skip this step)
    if 'year-month' in wb.sheetnames:
      wb.remove(wb['year-month'])

    wb.save(output_path)
    print(f"Batch {batch_number} saved to {output_path}")

print("Data inserted into new worksheets and saved.")


Batch 1 saved to /content/drive/MyDrive/Colab Notebooks/GSIS Arrears Automation/computed_output_sheets_batch1.xlsx
Batch 2 saved to /content/drive/MyDrive/Colab Notebooks/GSIS Arrears Automation/computed_output_sheets_batch2.xlsx
Data inserted into new worksheets and saved.
